In [1]:
import torch

import numpy as np

import matplotlib.pyplot as plt
import cv2

import json
import os
from os.path import join, split
import zipfile
from glob import glob

In [2]:
class config:
  data_path = "data"

In [47]:
class Video():
  def __init__(self, name:str) -> None:
    self.path = name
    self.image_paths = glob(f"{self.path}/**/**.jpg")
    self.label_path = glob(f"{self.path}/**.json")[0]

    self.class_names = {'player': 0,
  'referee': 1,
  'ball': 2,
  'Side line top': 3,
  'Middle line': 4,
  'Circle central': 5,
  'Big rect. left main': 6,
  'Big rect. left top': 7,
  'Circle left': 8,
  'Side line left': 9,
  'Small rect. left top': 10,
  'Small rect. left main': 11,
  'goalkeeper': 12,
  'Goal left post right': 13,
  'Goal left crossbar': 14,
  'Side line bottom': 15,
  'Small rect. left bottom': 16,
  'Goal left post left': 17,
  'Big rect. left bottom': 18,
  'Big rect. right main': 19,
  'Big rect. right top': 20,
  'Circle right': 21,
  'Side line right': 22,
  'Small rect. right main': 23,
  'Small rect. right top': 24,
  'Goal right post left': 25,
  'Goal right crossbar': 26,
  'Goal right post right': 27,
  'Small rect. right bottom': 28,
  'Big rect. right bottom': 29,
  'other':30}

  def __len__(self):
    return len(self.image_paths)

  def run_inference(self, img_path):
    img = np.array(Image.open(img_path))
    results = model(img)
    plt.imshow(draw_bboxes(results))

    

  def json2yolo(self):
      json_file = self.label_path
      with open(json_file, 'r') as f:
          data = json.load(f)

      all_classes = {}
      classes = []

      for i in data['annotations']:
        img_id = i['image_id'][4:]
        txt = ""
        if i['category_id'] == 5: # Pitch
          lines = i['lines']
          for j in lines:
            label = self.class_names[j]
            coords = lines[j]
            w = 1/1920
            h=1/1080
            for k in coords:
               x = k['x']/1920
               y = k['y']/1080
               txt += f"{label} {x} {y} {w} {h}\n"
        else: # Not Pitch
          label = self.class_names[i['attributes']['role']]
          box = i['bbox_image']
          x = box['x_center']/1920
          y = box['y_center']/1080
          w = box['w']/1920
          h = box['h']/1080

          txt += f"{label} {x} {y} {w} {h}\n"
        if not os.path.exists(join(os.path.split(json_file)[0],f"labels")):
          os.mkdir(join(os.path.split(json_file)[0],f"labels"))

        f = open(join(os.path.split(json_file)[0],f"labels/{img_id}.txt"), "a")
        f.write(txt)
        f.close()


  def __getitem__(self, idx):
    img_path = self.image_paths[idx]
    img_id = os.path.split(img_path)[-1]
    print(img_id)



In [48]:
all_clips = glob(f"{config.data_path}/**/**/")
print(f"{len(all_clips)} Total Clips")

164 Total Clips


In [49]:
for i in all_clips:
    vid = Video(i)
    path = (f"{split(vid.label_path)[0]}\\labels")
    try:
        if os.path.exists(path):
            for j in os.listdir(path):
                os.remove(join(i,j))
            os.remove(path)
    except:
        pass

In [50]:
for i in all_clips:
    vid = Video(i)
    vid.json2yolo()

In [51]:
for i in all_clips:
    folder_id = i[-4:-1]
    for j in os.listdir(join(i,"images")):
        try:
            os.rename(join(i,"images",j), join(i,"images",f"{folder_id}{j}"))
        except:
            pass
    for j in os.listdir(join(i,"labels")):
        try:
            os.rename(join(i,"labels",j), join(i,"labels",f"{folder_id}{j}"))
        except:
            pass

In [52]:
for i in all_clips:
    for j in os.listdir(join(i,"images")):

        path = join(i,"images",j).split("\\")
        og_path = join(i,"images",j)
        new_path = (f"datasets\\{path[1]}\\{path[3]}\\{path[4]}")
        if os.path.exists(og_path):
            os.rename(og_path, new_path)
    for j in os.listdir(join(i,"labels")):
        path = join(i,"labels",j).split("\\")
        og_path = join(i,"labels",j)
        new_path = (f"datasets\\{path[1]}\\{path[3]}\\{path[4]}")
        if os.path.exists(og_path):
            os.rename(og_path, new_path)

In [53]:
for label_file in glob(f"datasets\\train\\labels\\*"):
    with open(label_file, "r") as f:
        lines = f.readlines()
        for i in lines:
            if len(i)==0:
                print(label_file)

KeyboardInterrupt: 

In [ ]:

# with open("yourfile.txt", "w") as f:
#     for line in lines:
#         if line.strip("\n") != "nickname_to_delete":
#             f.write(line)

21
21
21
21
20
21
22
22
22
22
22
22
22
22
22
22
22
22
23
20
